## Trabalhando com o comando SELECT em Dataframes - Parte 2
Vamos continuar aprendendo a usar o comando SELECT em Spark Dataframes! Agora vamos explorar outros comandos como SelectExpr, limit, union e geração de amostras de dados

### Importando as bibliotecas
Nesta etapa iremos apenas importar todas as bibliotecas e funções necessárias para rodar o programa

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col

### Criando uma SparkSession
Por meio de uma SparkSession terei acesso ao SparkContext da minha aplicação.

In [5]:
# Inicia uma sparkSession
spark = SparkSession \
    .builder \
    .appName("Meu curso de pyspark") \
    .getOrCreate()

### Carregando dados a partir de um arquivo CSV
Nesta etapa estamos carregando os dados que utilizaremos neste notebook

In [7]:
dados = spark \
    .read.option("header","true") \
    .option("inferSchema","true") \
    .option("delimiter",";") \
    .format("csv") \
    .load("/FileStore/tables/bank_additional_full-3fd09.csv")

### Trabalhando com SQL Expressions
Expressões são úteis para realizar transformações nos dados usando SQL! Veja a seguir alguns exemplos básicos de expressões no Spark

In [9]:
# Sem usar SQL
dados.select(dados['age'] + 10).show(5)

# Usando SQL
dados.select(expr("age + 10")).show(5)

+----------+
(age + 10)|
+----------+
 66|
 67|
 47|
 50|
 66|
+----------+
only showing top 5 rows

+----------+
(age + 10)|
+----------+
 66|
 67|
 47|
 50|
 66|
+----------+
only showing top 5 rows

**Usando selectExpr para escrever consultas SQL**

In [11]:
dados.selectExpr("*", "(age > 40) as idade_maior_40").show(5)

+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+--------------+
age| job|marital| education|default|housing|loan| contact|month|day_of_week|duration|campaign|pdays|previous| poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed| y|idade_maior_40|
+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+--------------+
 56|housemaid|married| basic.4y| no| no| no|telephone| may| mon| 261| 1| 999| 0|nonexistent| 1.1| 93.994| -36.4| 4.857| 5191.0| no| true|
 57| services|married|high.school|unknown| no| no|telephone| may| mon| 149| 1| 999| 0|nonexistent| 1.1| 93.994| -36.4| 4.857| 5191.0| no| true|
 37| services|married|high.school| no| yes| no|telephone| may| mon| 226| 1| 999| 0|nonexistent| 1.1| 93.994| -36.4| 4.857| 5191.0| no| false|
 40| admin.|married| basic.6y| no| no| no|telephone| may| mon| 151| 1| 999| 0|nonexistent| 1.1| 93.994| -36.4| 4.857| 5191.0| no| false|
 56| services|married|high.school| no| no| yes|telephone| may| mon| 307| 1| 999| 0|nonexistent| 1.1| 93.994| -36.4| 4.857| 5191.0| no| true|
+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+--------------+
only showing top 5 rows

**Selecionando a maior idade com selectExpr**

In [13]:
dados.selectExpr("max(age)").show()

+--------+
max(age)|
+--------+
 98|
+--------+

**Utilizando palavras reservadas do Spark e caracteres especiais em nome de colunas**

In [15]:
dados.selectExpr("age as `idade com espaço`").show(2)

+----------------+
idade com espaço|
+----------------+
 56|
 57|
+----------------+
only showing top 2 rows

### Trabalhando com amostras de dados
Pode ser que você não queira trabalhar com todo o seu conjunto de dados. Isso pode ocorrer por diversos motivos, os mais comuns são:
* O seu dataframe é muito grande e você quer apenas usar uma parte dele para testar uma ideia
* Você está criando um modelo de machine learning e precisa dividir os dados entre treino e teste, por exemplo
<p>A seguir veremos algumas maneiras de trabalharmos com amostragem de dados no Spark

**Selecionando uma amostra de 10% dos dados**

In [18]:
seed = 2020
withReplacement = False
fraction = 0.1
dados.sample(withReplacement, fraction, seed).count()

Out[15]: 4210

**Dividindo os dados em amostras diferentes**

In [20]:
seed = 2020
dataFrames = dados.randomSplit([0.25, 0.75], seed)
print(dataFrames[0].count())
print(dataFrames[1].count())

10391
30797

### Unindo linhas
No exemplo anterior, dividimos o Dataframe em dois subconjuntos de dados. E se quisermos unir estes conjuntos novamente?<br>
Para isso, podemos usar o comando UNION

In [22]:
dados1 = dataFrames[0].union(dataFrames[1]).where(col('marital') == "married")
dados1.show(5)

+---+-----------+-------+-----------+-------+-------+----+--------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
age| job|marital| education|default|housing|loan| contact|month|day_of_week|duration|campaign|pdays|previous| poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed| y|
+---+-----------+-------+-----------+-------+-------+----+--------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
 21|blue-collar|married| basic.9y| no| no| yes|cellular| jul| wed| 89| 6| 999| 0|nonexistent| 1.4| 93.918| -42.7| 4.963| 5228.1| no|
 21|blue-collar|married| basic.9y| no| yes| no|cellular| jul| wed| 2078| 6| 999| 0|nonexistent| 1.4| 93.918| -42.7| 4.963| 5228.1|yes|
 22| admin.|married|high.school| no| no| yes|cellular| apr| fri| 108| 2| 999| 1| failure| -1.8| 93.075| -47.1| 1.405| 5099.1| no|
 22|blue-collar|married| basic.9y| no| yes| no|cellular| jul| thu| 203| 1| 999| 0|nonexistent| 1.4| 93.918| -42.7| 4.962| 5228.1| no|
 22|blue-collar|married| basic.9y| no| yes| no|cellular| jul| thu| 448| 1| 999| 0|nonexistent| 1.4| 93.918| -42.7| 4.962| 5228.1| no|
+---+-----------+-------+-----------+-------+-------+----+--------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
only showing top 5 rows

### Limitando o resultado
Utilize o comando LIMIT para limitar o número de linhas que serão retornadas na sua consulta

In [24]:
dados1.limit(5)

Out[24]: DataFrame[age: int, job: string, marital: string, education: string, default: string, housing: string, loan: string, contact: string, month: string, day_of_week: string, duration: int, campaign: int, pdays: int, previous: int, poutcome: string, emp.var.rate: double, cons.price.idx: double, cons.conf.idx: double, euribor3m: double, nr.employed: double, y: string]

### Obrigado!
Quer construir uma carreira em Data Science? Acesse meu blog pessoal em https://www.hackinganalytics.com/